<center>
<img src="https://unskript.com/assets/favicon.png" alt="unSkript.com" width="100" height="100"/> 
<h1> unSkript Runbooks </h1>

<div class="alert alert-block alert-success">
    <h3> Objective</h3> <br>
    <b>To add a new IAM user using unSkript actions.</b>
</div>
</center>
<br>

<center><h2>Add New IAM User</h2></center>

# Steps Overview
   1)[Create IAM User](#1)</br>
   2)[Create login profile](#2)</br>
   3)[Check the caller identity](#3)</br>
   4)[Post slack message](#4)

In [5]:
tag_key = "Name"
tag_value = username

<h3 id="Creating-an-IAM-User"><a id="1" target="_self" rel="nofollow"></a>Creating an IAM User</h3>
<p>Here we will use unSkript <strong>Create New IAM User</strong> action. This action creates an IAM user in AWS and assigns the given tag to the user.</p>
<blockquote>
<p>Input parameters: <code>user_name</code>, <code>tag_key</code>, <code>tag_value</code></p>
</blockquote>
<blockquote>
<p>Output variable: <code>user_details</code></p>
</blockquote>

In [ ]:
##  Copyright (c) 2021 unSkript, Inc
##  All rights reserved.
##
from typing import List, Dict
from pydantic import BaseModel, Field
from botocore.exceptions import ClientError
import pprint


from beartype import beartype
@beartype
def aws_create_iam_user_printer(output):
    if output is None:
        return
    pprint.pprint(output)


@beartype
def aws_create_iam_user(handle, user_name: str, tag_key: str, tag_value: str) -> Dict:
    """aws_create_iam_user Creates new IAM User.

        :type handle: object
        :param handle: Object returned by the task.validate(...) method

        :type user_name: string
        :param user_name: Name of new IAM User.

        :type tag_key: string
        :param tag_key: Tag Key assign to new User.

        :type tag_value: string
        :param tag_value: Tag Value assign to new User.

        :rtype: Dict with the stopped instances state info.
    """

    ec2Client = handle.client("iam")
    result = {}
    try:
        response = ec2Client.create_user(
            UserName=user_name,
            Tags=[
                {
                    'Key': tag_key,
                    'Value': tag_value
                }])
        result = response
    except ClientError as error:
        if error.response['Error']['Code'] == 'EntityAlreadyExists':
            result = error.response
        else:
            result = error.response

    return result


task = Task(Workflow())
task.configure(printOutput=True)
task.configure(inputParamsJson='''{
    "tag_key": "tag_key",
    "tag_value": "tag_value",
    "user_name": "username"
    }''')
task.configure(outputName="user_details")

(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_create_iam_user, lego_printer=aws_create_iam_user_printer, hdl=hdl, args=args)

<h3 id="Create-login-Profile"><a id="2" target="_self" rel="nofollow"></a>Create login Profile</h3>
<p>This action only executes when step 1 successfully creates a user. In this action, we will pass the newly created username and temporary password, which will create an user profile for the user in AWS.</p>
<blockquote>
<p>Input parameters: <code>user_name</code>, <code>password</code></p>
</blockquote>
<blockquote>
<p>Output variable: <code>profile_details</code></p>
</blockquote>

In [ ]:
##
##  Copyright (c) 2021 unSkript, Inc
##  All rights reserved.
##
from pydantic import BaseModel, Field
from typing import Dict
from botocore.exceptions import ClientError
import pprint


from beartype import beartype
@beartype
def aws_create_user_login_profile_printer(output):
    if output is None:
        return
    pprint.pprint(output)


@beartype
def aws_create_user_login_profile(handle, user_name: str, password: str) -> Dict:
    """aws_create_user_login_profile Create login profile for IAM User.

        :type handle: object
        :param handle: Object returned by the task.validate(...) method.

        :type user_name: string
        :param user_name: Name of new IAM User.

        :type password: string

        :rtype: Dict with the Profile Creation status info.
    """

    ec2Client = handle.client("iam")
    result = {}
    try:
        response = ec2Client.create_login_profile(
            UserName=user_name,
            PasswordResetRequired=True)

        result = response
    except ClientError as error:
        if error.response['Error']['Code'] == 'EntityAlreadyExists':
            result = error.response
        else:
            result = error.response

    return result


task = Task(Workflow())
task.configure(printOutput=True)
task.configure(inputParamsJson='''{
    "password": "password",
    "user_name": "username"
    }''')
task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "'User' in UserInfo",
    "condition_result": true
    }''')
task.configure(outputName="profile_details")

(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_create_user_login_profile, lego_printer=aws_create_user_login_profile_printer, hdl=hdl, args=args)

<h3 id="Check-Caller-Identity"><a id="3" target="_self" rel="nofollow"></a>Check Caller Identity</h3>
<p>Here we will use unSkript <strong>Get Caller Identity Action</strong> action. These Action does not take any inputs. shows the caller's identity for the current user.</p>
<blockquote>
<p>Output variable: <code>caller_details</code></p>
</blockquote>

In [5]:
##  Copyright (c) 2021 unSkript, Inc
##  All rights reserved.
##
from typing import List, Dict
from pydantic import BaseModel, Field
import pprint


from beartype import beartype
@beartype
def aws_get_caller_identity_printer(output):
    if output is None:
        return
    pprint.pprint(output)


@beartype
def aws_get_caller_identity(handle) -> Dict:
    ec2Client = handle.client('sts')
    response = ec2Client.get_caller_identity()

    return response


task = Task(Workflow())
task.configure(printOutput=True)
task.configure(outputName="caller_details")
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_get_caller_identity, lego_printer=aws_get_caller_identity_printer, hdl=hdl, args=args)

<h3 id="Post-Slack-Message"><a id="4" target="_self" rel="nofollow"></a>Post Slack Message</h3>
<p>Here we will use unSkript <strong>Post Slack Message</strong> action. These actions send a message on the Slack channel with the newly created username.</p>
<blockquote>
<p>Input parameters: <code>channel</code>, <code>message</code></p>
</blockquote>
<blockquote>
<p>Output variable: <code>send_status</code></p>
</blockquote>

In [6]:
##
# Copyright (c) 2021 unSkript, Inc
# All rights reserved.
##

import pprint

from pydantic import BaseModel, Field
from slack_sdk import WebClient
from slack_sdk.errors import SlackApiError

pp = pprint.PrettyPrinter(indent=2)


from beartype import beartype
@beartype
def slack_post_message_printer(output):
    if output is not None:
        pprint.pprint(output)
    else:
        return


@beartype
def slack_post_message(
        handle: WebClient,
        channel: str,
        message: str) -> str:

    try:
        response = handle.chat_postMessage(
            channel=channel,
            text=message)
        return f"Successfuly Sent Message on Channel: #{channel}"
    except SlackApiError as e:
        pp.pprint(
            f"Failed sending message to slack channel {channel}, Error: {e.response['error']}")
        if e.response['error'] == 'channel_not_found':
            raise Exception('Channel Not Found')
        elif e.response['error'] == 'duplicate_channel_not_found':
            raise Exception('Channel associated with the message_id not valid')
        elif e.response['error'] == 'not_in_channel':
            raise Exception('Cannot post message to channel user is not in')
        elif e.response['error'] == 'is_archived':
            raise Exception('Channel has been archived')
        elif e.response['error'] == 'msg_too_long':
            raise Exception('Message text is too long')
        elif e.response['error'] == 'no_text':
            raise Exception('Message text was not provided')
        elif e.response['error'] == 'restricted_action':
            raise Exception('Workspace preference prevents user from posting')
        elif e.response['error'] == 'restricted_action_read_only_channel':
            raise Exception('Cannot Post message, read-only channel')
        elif e.response['error'] == 'team_access_not_granted':
            raise Exception('The token used is not granted access to the workspace')
        elif e.response['error'] == 'not_authed':
            raise Exception('No Authtnecition token provided')
        elif e.response['error'] == 'invalid_auth':
            raise Exception('Some aspect of Authentication cannot be validated. Request denied')
        elif e.response['error'] == 'access_denied':
            raise Exception('Access to a resource specified in the request denied')
        elif e.response['error'] == 'account_inactive':
            raise Exception('Authentication token is for a deleted user')
        elif e.response['error'] == 'token_revoked':
            raise Exception('Authentication token for a deleted user has been revoked')
        elif e.response['error'] == 'no_permission':
            raise Exception('The workspace toekn used does not have necessary permission to send message')
        elif e.response['error'] == 'ratelimited':
            raise Exception('The request has been ratelimited. Retry sending message later')
        elif e.response['error'] == 'service_unavailable':
            raise Exception('The service is temporarily unavailable')
        elif e.response['error'] == 'fatal_error':
            raise Exception('The server encountered catostrophic error while sending message')
        elif e.response['error'] == 'internal_error':
            raise Exception('The server could not complete operation, likely due to transietn issue')
        elif e.response['error'] == 'request_timeout':
            raise Exception('Sending message error via POST: either message was missing or truncated')
        else:
            raise Exception(f'Failed Sending Message to slack channel {channel} Error: {e.response["error"]}')

    except Exception as e:
        print("\n\n")
        pp.pprint(
            f"Failed sending message to slack channel {channel}, Error: {e.__str__()}")
        return f"Unable to send message on {channel}"


task = Task(Workflow())
task.configure(printOutput=True)
task.configure(inputParamsJson='''{
    "channel": "Channel_Name",
    "message": "\\"New IAM user {}\\".format(user_name)"
    }''')

task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "'User' in UserInfo and not channel",
    "condition_result": true
    }''')
task.configure(outputName="send_status")

(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(slack_post_message, lego_printer=slack_post_message_printer, hdl=hdl, args=args)

### Conclusion
In this Runbook, we demonstrated the use of unSkript's AWS and slack actions to perform AWS create new IAM user, login profile and also show the caller identity of the user. On Success, post a message on the slack channel about the User creation. To view the full platform capabilities of unSkript please visit https://us.app.unskript.io